# jiu-jitsu-auto-scoring
*This notebook is a **WIP**!*


In [27]:
# prompt: mount drive and clone the repository in the current branch, the organization is called jiu-jitsu-auto-scoring and the repository is called model
from google.colab import drive

BRANCH = "5-try-out-vitpose" # Write the name of the branch you're working in here

!git clone -q https://github.com/jiu-jitsu-auto-scoring/model.git
%cd /content/model
!git pull
!git checkout $BRANCH

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [28]:
!git clone https://github.com/ViTAE-Transformer/ViTPose.git
%cd ViTPose

!pip install -r requirements.txt
!pip install -e .

!pip install supervision # for visualizing keypoints


NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

Create a folder Model/VitPose/Checkpoints. Upload the required checkpoint into that folder

In [ ]:
!mkdir -p checkpoints

In [ ]:
import torch
import requests
import numpy as np

from PIL import Image

from transformers import AutoProcessor, RTDetrForObjectDetection, VitPoseForPoseEstimation


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
url = "http://images.cocodataset.org/val2017/000000000139.jpg"
image = Image.open(requests.get(url, stream=True).raw)

Stage 1. Detect humans on the image


In [ ]:
# You can choose any detector of your choice
person_image_processor = AutoProcessor.from_pretrained("PekingU/rtdetr_r50vd_coco_o365")
person_model = RTDetrForObjectDetection.from_pretrained("PekingU/rtdetr_r50vd_coco_o365", device_map=device)

inputs = person_image_processor(images=image, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = person_model(**inputs)

results = person_image_processor.post_process_object_detection(
    outputs, target_sizes=torch.tensor([(image.height, image.width)]), threshold=0.3
)
result = results[0]  # take first image results

# Human label refers 0 index in COCO dataset
person_boxes = result["boxes"][result["labels"] == 0]
person_boxes = person_boxes.cpu().numpy()

# Convert boxes from VOC (x1, y1, x2, y2) to COCO (x1, y1, w, h) format
person_boxes[:, 2] = person_boxes[:, 2] - person_boxes[:, 0]
person_boxes[:, 3] = person_boxes[:, 3] - person_boxes[:, 1]


Stage 2. Detect keypoints for each person found

In [29]:
image_processor = AutoProcessor.from_pretrained("usyd-community/vitpose-base-simple")
model = VitPoseForPoseEstimation.from_pretrained("usyd-community/vitpose-base-simple", device_map=device)

inputs = image_processor(image, boxes=[person_boxes], return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)

pose_results = image_processor.post_process_pose_estimation(outputs, boxes=[person_boxes])
image_pose_result = pose_results[0]  # results for first image

In [30]:


import supervision as sv

xy = torch.stack([pose_result['keypoints'] for pose_result in image_pose_result]).cpu().numpy()
scores = torch.stack([pose_result['scores'] for pose_result in image_pose_result]).cpu().numpy()

key_points = sv.KeyPoints(
    xy=xy, confidence=scores
)

edge_annotator = sv.EdgeAnnotator(
    color=sv.Color.GREEN,
    thickness=1
)
vertex_annotator = sv.VertexAnnotator(
    color=sv.Color.RED,
    radius=2
)
annotated_frame = edge_annotator.annotate(
    scene=image.copy(),
    key_points=key_points
)
annotated_frame = vertex_annotator.annotate(
    scene=annotated_frame,
    key_points=key_points
)

ModuleNotFoundError: No module named 'supervision'